# 3. Autoencoders (AE) en MNIST

**Importación de módulos necesarios**

In [16]:
from keras.datasets import mnist
import numpy as np
from keras.layers import Input, Dense
from keras.models import Model
from keras.models import load_model
from keras.optimizers import SGD, adadelta
import warnings
import matplotlib.pyplot as plt

**3.0 Creación de conjuntos de datos a utilizar**

En primer lugar, se crean los conjuntos de datos a ser utilizados en la resolución del problema. Para ello, el dataset completo de imágenes es obtenido desde el repositorio de *keras*. Posteriormente, las matrices que contienen las imágenes, *x_train* y *x_test*, son escaladas en base a la intensidad máxima de píxel y luego reorganizadas como un vector cada una.

Además, se crea el conjunto de validación que será usado más adelante, extrayendo para ello los últimos 5.000 registros del conjunto de entrenamiento.

In [3]:
# Se cargan conjutos de entrenamiento y de prueba
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Se normalizan conjuntos de datos en base a intensidad máxima de pixel
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

# Se transforman las imágenes de ambos conjuntos a vectores
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

# Se define conjunto de validación y se reestructura conjunto de entrenamiento
x_val = x_train[5000:, :]
x_train = x_train[:5000, :]
y_val = y_train[5000:]
y_train = y_train[:5000]

# 3.1 Reducción de dimensionalidad

En esta sección, se hará uso de diversos AE's como reductores de dimensionalidad, esperando mejorar el rendimiento de modelos convolucionales, los cuales recibirán como inputs las imágenes "reducidas" y serán entrenados para construir un clasificador de dígitos. 

**3.1.1 AE con una capa escondida**

Se comienza construyendo un AE simple, compuesto únicamente por una capa oculta (enconder) y una capa de salida (decoder). En principio, el enconder utilizará la función de activación sigmoide, para luego experimentar con ReLU, al igual que el decoder. Se usará *adadelta* como método de entrenamiento y *binary crossentropy* como función de pérdida. Para la implementación del AE, se construye la función *simple_AE*, que recibe como parámetro el nivel de compresión deseado y la función de activación del encoder. Como salida, se generan archivos que registran el porcentaje de compresión obtenido y el error de reconstrucción de cada configuración.

El enconder estará integrado por d' neuronas, con d' en {2, 8, 32, 64}. Esto quiere decir que el input original, de 784 dimensiones, será comprimido en un vector de d' dimensiones.

El decoder en cambio, estára compuesto, naturalmente, por 784 neuronas, pues se busca restaurar la estructura inicial del input recibido.

In [4]:
# Implementación de función simple_AE; d_ : compresión deseada; act_func: función de activación encoder
def simple_AE(d_, act_functions):
    # Se determina el tipo de input a ser recibido por el AE
    input_img = Input(shape=(784,))
    # "encoded" es la versión codificada del input
    encoded = Dense(d_, activation=act_functions[0])(input_img)
    # "decoded" es la reconstrucción del input codificado
    decoded = Dense(784, activation=act_functions[1])(encoded)
    # Se genera AE a partir de capas anteriores, el cual mapea un input hacia su reconstrucción
    autoencoder = Model(input=input_img, output=decoded)
    # Se definen método de entrenamiento y función de pérdida
    autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
    # Se entrena AE
    history = autoencoder.fit(x_train, x_train,
              epochs=50,
              batch_size=256,
              shuffle=True,
              verbose=0,
              validation_data=(x_val, x_val))
    # Se guardan porcentaje de compresión y error de reconstrucción en un archivo
    autoencoder.save('basic_autoenconder_784x' + str(d_) + act_functions[0] + '_' + act_functions[1] + '.h5')

Así, se procede a generar AE's para d' dimensiones y utilizando la función ReLU y/o sigmoide en el enconder y/o el decoder.

In [17]:
# Valores posibles para la dimensión del input comprimido
dimensions = [2, 8, 32, 64]
# Funciones de activación posibles
activations = [('sigmoid', 'sigmoid'), ('relu', 'sigmoid'), ('sigmoid', 'relu')]

In [5]:
warnings.filterwarnings('ignore')

# Se generan todos los AE's posibles
for dimension in dimensions:
    for activation in activations:
        simple_AE(dimension, activation)

Generados los archivos, se procede a leerlos para determinar el rendimiento de cada una de las configuraciones. 

In [6]:
for activation in activations:
    print('Encoder: ' + activation[0] + ' | ' + 'Decoder: ' + activation[1])
    for dimension in dimensions:
        autoencoder = load_model('basic_autoenconder_784x' + str(dimension) + activation[0] + '_' + activation[1] + '.h5')
        compression = float(dimension) / 784.
        error = autoencoder.evaluate(x_test, x_test, batch_size=10, verbose=0)
        print('Compresion: ' + str(compression) + ' %' + ' | ' + 'Error: ' + str(error))
    print('\n')

Encoder: sigmoid | Decoder: sigmoid
Compresion: 0.00255102040816 % | Error: 0.316933655649
Compresion: 0.0102040816327 % | Error: 0.270845110208
Compresion: 0.0408163265306 % | Error: 0.263548591167
Compresion: 0.0816326530612 % | Error: 0.214989810884


Encoder: relu | Decoder: sigmoid
Compresion: 0.00255102040816 % | Error: 0.248252324775
Compresion: 0.0102040816327 % | Error: 0.182951631218
Compresion: 0.0408163265306 % | Error: 0.121280142166
Compresion: 0.0816326530612 % | Error: 0.0979144032449


Encoder: sigmoid | Decoder: relu
Compresion: 0.00255102040816 % | Error: 0.909125794262
Compresion: 0.0102040816327 % | Error: 0.766526055366
Compresion: 0.0408163265306 % | Error: 0.624800803095
Compresion: 0.0816326530612 % | Error: 0.544563655525




En consecuencia, se observa que no es una buena práctica utilizar la función de activación relu en el decoder, pues independiente de la compresión utilizada, el error de reconstrucción es superior o igual al 54%.
Por otro lado, los errores más bajos se consiguen al usar relu en el encoder y sigmoide en el decoder, para toda compresión posible.
Es importante notar que si la compresión es demasiado alta (es decir, si el input codificado posee una baja dimensionalidad), el error aumenta, como ocurre para d' = 2.

**3.1.2 Comparando reconstrucciones de imágenes**

Se desea comparar la "calidad" de reconstrucción de los autoencoders implementados en la sección anterior sobre algunas imágenes del conjunto de pruebas. Considerando que al utilizar la función de activación ReLU en el encoder y sigmoide en en el decoder se obtienen los errores de reconstrucción más bajos, se recuperarán los cuatro modelos que hacen uso de esta configuación.

In [15]:
# Se recuperan autoencoders de encoder ReLU y decoder sigmoide, para cada d' posible
autoencoder_784x2 = load_model('basic_autoenconder_784x2relu_sigmoid.h5')
autoencoder_784x8 = load_model('basic_autoenconder_784x8relu_sigmoid.h5')
autoencoder_784x32 = load_model('basic_autoenconder_784x32relu_sigmoid.h5')
autoencoder_784x64 = load_model('basic_autoenconder_784x64relu_sigmoid.h5')

autoencoders = {}
autoencoders['autoencoder_784x2'] = autoencoder_784x2
autoencoders['autoencoder_784x8'] = autoencoder_784x8
autoencoders['autoencoder_784x32'] = autoencoder_784x32
autoencoders['autoencoder_784x64'] = autoencoder_784x64

Luego, se continúa con la implementación de forma separada de un encoder y un decoder para cada d' posible. Para ello, se crean las funciones *make_encoder* y *make_autoencoder*, que reciben como parámetro d' y la función de activación deseada, en cada caso.

In [14]:
# Función para creación de encoder
def make_encoder(d_, act_function):
    input_img = Input(shape=(784,))
    encoded = Dense(d_, activation=act_function)(input_img)
    # Se define encoder, el cual mapea un input hacia su versión codificada
    encoder = Model(input=input_img, outputs=encoded)
    return encoder

# Función para creación de encoder
def make_decoder(d_, act_function, autoencoder):
    # encoded_input: Capa que permite la entrada de un input codificado en 32 dimensiones
    encoded_input = Input(shape=(d_,))
    # Se recupera última capa de autoencoder
    decoder_layer = autoencoder.layers[-1]
    # Se define decoder, el cual mapea un input comprimido hacia su versión "original"
    decoder = Model(input=encoded_input, output=decoder_layer(encoded_input))
    return decoder

Así, para cada d' posible, se estudian dos imágenes del conjunto de pruebas. Por cada una, se comparan su versión original y su reconstrucción. 

In [ ]:
encoder_act_function = 'relu'
decoder_act_function = 'sigmoid'
n = 2 # Cantidad de imágenes a estudiar por cada d'

for dimension in dimensions:
    print("Imagenes para d' = ", dimension)
    autoencoder = autoencoders['autoencoder_784_x' + dimension]
    encoder = make_encoder(dimension, encoder_act_function)
    decoder = make_decoder(dimension, decoder_act_function, autoencoder)
    encoded_test = encoder.predict(x_test)
    decoded_test = decoder.predict(encoded_test)
    plt.figure(figsize=(20, 4))
    for in in range(n):
        ax = plt.subplot(2, n, i + 1)
        plt.imshow(x_test)